# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [ ]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install yfinance --upgrade

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
from openai import OpenAI
import yfinance as yf
import os
import time

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [ ]:
K = 25

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/FZ4025 - Fintech e innovación digital en finanzas/Week 4/"
os.chdir(DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_news = pd.read_csv('df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [ ]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [ ]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [ ]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [ ]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [ ]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [ ]:
# CODE HERE
# Use as many coding cells as you need
from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}
    ]
)

print(response.choices[0].message.content)

As the moonlight shimmered on the tranquil meadow, a gentle unicorn named Luna danced among the stars, sprinkling dreams of magic and wonder into the hearts of all who slept nearby.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [ ]:
# CODE HERE
# Use as many coding cells as you need
unique_tickers = df_news['TICKER'].unique()
display(unique_tickers[:5])

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'], dtype=object)

In [ ]:
ticker_list = []
company_name_list = []
sector_list = []
industry_list = []

for i, ticker in enumerate(unique_tickers):
  ticker_obj = yf.Ticker(ticker)
  ticker_info = ticker_obj.info
  company_name = ticker_info.get('longName', 'N/A')
  sector = ticker_info.get('sector', 'N/A')
  industry = ticker_info.get('industry', 'N/A')

  ticker_list.append(ticker)
  company_name_list.append(company_name)
  sector_list.append(sector)
  industry_list.append(industry)

  time.sleep(0.1)

In [ ]:
df_meta = pd.DataFrame({
    'TICKER': ticker_list,
    'COMPANY_NAME': company_name_list,
    'SECTOR': sector_list,
    'INDUSTRY': industry_list
})

display(df_meta.head(10))

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
5,ADBE,Adobe Inc.,Technology,Software - Application
6,AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors
7,AES,The AES Corporation,Utilities,Utilities - Diversified
8,AFL,Aflac Incorporated,Financial Services,Insurance - Life
9,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [ ]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [ ]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [ ]:
# CODE HERE
# Use as many coding cells as you need

class RAG:
    def __init__(self, vector_store, openai_client, k=K):
        self.vector_store = vector_store
        self.client = openai_client
        self.k = k

    def query(self, question):
        docs = self.vector_store.search(question, self.k)

        context = ""

        for i, (doc, meta, score) in enumerate(docs):
            context += f"""Article {i+1}:
              Content: {doc}
              Ticker: {meta['TICKER']}
              Date: {meta['PUBLICATION_DATE']}
              Relevance Score: {score:.3f}
              """

        prompt = f"""Question: {question}

        Based on these financial news articles, provide a comprehensive answer.

        Context Articles:
        {context}

        Answer:"""

        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a financial analyst. Answer based on the provided news articles. Be specific and cite details."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=800
        )

        answer = response.choices[0].message.content
        doc_info = [(doc, meta, score) for doc, meta, score in docs]

        return answer, doc_info

    def display_results(self, question, answer, docs):
        output = f"# Question: {question}\n\n## Answer:\n{answer}\n\n## Sources:\n"

        for i, (doc, meta, score) in enumerate(docs[:5], 1):
            output += f"**{i}.** {doc[:200]}...\n*{meta['PUBLICATION_DATE']} | {meta['TICKER']} | Score: {score:.3f}*\n\n"

        print_markdown(output)

In [ ]:
rag = RAG(faiss_store, client)

In [ ]:
def test_rag_questions(rag, questions_list, category_name):
    print(f"Testing {category_name} Questions:")

    for question in questions_list:
        answer, docs = rag.query(question)
        rag.display_results(question, answer, docs)

        print("-" * 80)

In [ ]:
test_rag_questions(rag, questions_topic, "Topic")

Testing Topic Questions:


# Question: What are the major concerns expressed in financial news about inflation?

## Answer:
The financial news articles highlight several major concerns regarding inflation, which can be summarized as follows:

1. **Persistent Inflation Risks**: Article 1 discusses the Federal Reserve's May policy meeting, where officials expressed growing concerns about persistent inflation in the U.S. This indicates that inflation is not only a current issue but may continue to pose risks to the economy, potentially leading to an economic slowdown.

2. **Food Inflation and Monetary Policy**: Article 2 emphasizes that rising food inflation is dampening hopes for a rate cut by the Federal Reserve. This suggests that inflationary pressures, particularly in essential goods like food, are significant enough to influence central bank policy decisions, which could have broader implications for economic growth and consumer spending.

3. **Impact on Consumer Prices**: Article 6 notes that creeping inflation and new tariffs have caused grocery bills to skyrocket, affecting everyday consumers. The mention of specific items such as orange juice, eggs, and chicken breasts highlights how inflation is impacting basic necessities, which can strain household budgets and alter consumer behavior.

4. **Economic Outlook and Earnings**: Articles 15 and 19 point out that uncertainty about the macroeconomic environment, driven by inflation concerns, is affecting earnings outlooks. Analysts are cutting estimates due to the unpredictable nature of inflation and its potential impact on consumer spending and corporate profits.

5. **Geopolitical and Fiscal Concerns**: Article 14 discusses how persistent inflation concerns are intertwined with geopolitical tensions and the U.S. national debt, which has surpassed $36 trillion. This context suggests that inflation is not just a domestic issue but is also influenced by global factors and fiscal sustainability, leading investors to seek stability in hard assets like gold.

6. **Market Reactions and Investor Sentiment**: Articles 9 and 13 indicate that market behaviors, including stock performance and bond yields, are reflecting concerns about inflation and its implications for monetary policy. The commentary from Federal Reserve officials is closely watched as it provides insights into how inflation is shaping their decisions, which in turn affects investor sentiment and market dynamics.

In summary, the major concerns about inflation expressed in the financial news revolve around its persistence, impact on essential goods, influence on monetary policy, effects on corporate earnings, and broader economic implications tied to geopolitical and fiscal factors. These elements collectively paint a picture of a challenging economic landscape influenced heavily by inflationary pressures.

## Sources:
**1.** Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown....
*2025-05-29 | BLK | Score: 0.577*

**2.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*2025-05-31 | TSLA | Score: 0.492*

**3.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*2025-05-31 | NVDA | Score: 0.492*

**4.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*2025-05-31 | LULU | Score: 0.492*

**5.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*2025-05-31 | AVGO | Score: 0.492*



--------------------------------------------------------------------------------


# Question: How is investor sentiment described in recent financial headlines?

## Answer:
Investor sentiment in recent financial headlines is characterized by a mix of bullish optimism and cautious skepticism. Several articles highlight that while Wall Street analysts are generally optimistic about certain stocks, there are underlying concerns that warrant caution.

1. **Bullish Outlook with Caution**: Many articles report that Wall Street is overwhelmingly bullish on various stocks, suggesting significant upside potential based on ambitious price targets. For instance, Article 1 notes that Wall Street has set ambitious price targets for certain stocks, indicating attractive upside potential, but also emphasizes the need for skepticism due to institutional pressures that may lead to overly optimistic forecasts. Similarly, Article 5 mentions that analysts rarely issue sell ratings, which can inflate perceived stock values.

2. **Headwinds and Bearish Sentiment**: On the flip side, there are numerous mentions of stocks facing headwinds and bearish forecasts. Articles like Article 4 and Article 12 highlight that when Wall Street turns bearish on a stock, it is significant because such ratings are rare. Analysts often hesitate to issue negative ratings due to potential repercussions on their firms' business relationships, particularly in areas like M&A advisory. This suggests a cautious sentiment among investors regarding certain stocks that are underperforming or facing challenges.

3. **Market Performance and Catalysts**: Some articles point out that specific stocks have outperformed the market recently due to positive catalysts such as new product launches or favorable news coverage (e.g., Articles 2, 6, and 11). This indicates a segment of investor sentiment that is driven by excitement and momentum, particularly in stocks that are gaining traction among retail investors, possibly influenced by social media.

4. **Skepticism Towards Analyst Ratings**: There is a recurring theme of skepticism regarding analyst ratings and forecasts. Articles frequently remind readers to take analysts' bullish forecasts with a grain of salt, as the motivations behind these ratings can be influenced by the analysts' desire to maintain good relationships with the companies they cover (e.g., Articles 3, 8, and 23). This skepticism reflects a more cautious investor sentiment, urging investors to critically evaluate the underlying fundamentals of the stocks rather than relying solely on analyst recommendations.

In summary, the current investor sentiment is a blend of optimism for certain stocks driven by positive developments and skepticism regarding the reliability of analyst forecasts, especially in light of bearish trends for some stocks. Investors are advised to remain vigilant and consider both the bullish narratives and the potential risks associated with market dynamics.

## Sources:
**1.** 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to rem...
*2025-05-26 | KMX | Score: 0.612*

**2.** 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new pro...
*2025-05-20 | MCHP | Score: 0.598*

**3.** 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying ret...
*2025-05-06 | MPWR | Score: 0.589*

**4.** 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue g...
*2025-05-21 | DRI | Score: 0.577*

**5.** 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. How...
*2025-05-23 | RVTY | Score: 0.570*



--------------------------------------------------------------------------------


# Question: What role is artificial intelligence playing in recent finance-related news stories?

## Answer:
Artificial intelligence (AI) is playing a significant and multifaceted role in the finance sector, as highlighted by recent news articles. Here are some key insights based on the articles:

1. **Integration of AI in Lending and Financial Services**: Jack Henry & Associates has integrated AI-driven lending technology with Algebrik, showcasing how AI is being utilized to enhance productivity and reduce human error in financial transactions (Article 1). This integration reflects a broader trend where financial institutions are leveraging AI to streamline operations and improve customer experiences.

2. **Investment and Stock Performance**: Major companies like Meta Platforms are making substantial investments in AI applications, which are expected to significantly impact their stock valuations. Analysts suggest that Meta's stock is currently undervalued based on its AI investments, indicating a potential for growth if these investments yield positive results (Article 2). Similarly, hedge fund managers are buying stocks like Palantir and Upstart, which are anticipated to see substantial upside due to their AI capabilities in data analytics and credit risk assessment (Article 3).

3. **Market Dynamics and Stock Volatility**: The performance of AI stocks has been mixed, with companies like Palantir experiencing significant gains despite market volatility. Palantir's AI platform has been successful in both government and commercial sectors, which has contributed to its stock price increase of 63% in 2025 (Article 4). However, there are concerns about overvaluation, as seen in the case of Palantir, where its stock is considered expensive despite strong sales growth (Article 9).

4. **AI's Role in Financial Operations**: Companies like Intuit are leveraging AI to enhance customer experiences in tax preparation, which not only improves efficiency but also boosts revenue (Article 21). This demonstrates how AI can transform traditional financial services by automating complex processes and providing better customer support.

5. **Challenges and Strategic Focus**: Some companies, such as Salesforce, are facing challenges due to an overemphasis on AI at the expense of other business areas. Analysts have noted that Salesforce's heavy focus on AI has led to a deceleration in other parts of its business, suggesting that a balanced approach is necessary for sustainable growth (Article 7).

6. **Broader Market Impact**: The rise of AI is influencing the entire tech sector, with companies like Nvidia seeing increased demand for AI-related products, which in turn boosts the stock prices of related companies (Article 8). This trend indicates that AI is not just a niche market but a driving force across various sectors, including finance.

7. **Future Projections**: The AI data analytics market is projected to grow significantly, potentially exceeding $1 trillion by 2033 (Article 9). This growth potential is attracting investments and shaping strategic decisions across the finance and tech industries.

In summary, AI is reshaping the finance landscape by enhancing operational efficiencies, driving investment strategies, influencing stock performance, and presenting both opportunities and challenges for companies. The integration of AI technologies is becoming essential for financial institutions aiming to stay competitive and meet evolving consumer demands.

## Sources:
**1.** Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at wher...
*2025-03-17 | JKHY | Score: 0.697*

**2.** This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy busin...
*2025-05-31 | META | Score: 0.626*

**3.** Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in th...
*2025-05-31 | PLTR | Score: 0.619*

**4.** Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stoc...
*2025-05-31 | PLTR | Score: 0.618*

**5.** 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose ...
*2025-05-29 | NFLX | Score: 0.580*



--------------------------------------------------------------------------------


In [ ]:
test_rag_questions(rag, questions_company, "Company")

Testing Company Questions:


# Question: How is Microsoft being portrayed in news stories about artificial intelligence?

## Answer:
In recent news articles, Microsoft is portrayed as a significant player in the artificial intelligence (AI) landscape, particularly in the context of its investments and collaborations. The company is highlighted as one of the major innovators in the AI sector, alongside other tech giants like Google and NVIDIA. For instance, an article discussing the growth of the global AI market notes that Microsoft is leading key innovations, particularly in areas such as edge AI and AI chips, which are crucial for real-time processing and tailored applications (Article 6).

Moreover, Microsoft is mentioned in the context of its partnerships with other companies, such as ServiceNow, where it is involved in a "swarm of AI deals" that are expected to boost ServiceNow's stock performance. This collaboration indicates Microsoft's strategic focus on leveraging AI to enhance its business and that of its partners (Article 21).

Additionally, Microsoft's stock is included in discussions about the broader AI market dynamics, where analysts are optimistic about its potential to capitalize on the growing demand for AI technologies. This optimism is reflected in the context of AI stocks facing a "Show Me" moment, where companies need to demonstrate tangible revenue generation from their AI initiatives (Article 10).

Overall, Microsoft is portrayed positively in these articles, emphasizing its role as a leader in AI innovation and its strategic partnerships that position it well for future growth in the AI sector. The company's efforts in AI are seen as critical to its ongoing success and competitiveness in the tech industry.

## Sources:
**1.** This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy busin...
*2025-05-31 | META | Score: 0.543*

**2.** How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. L...
*2025-05-29 | CRM | Score: 0.505*

**3.** Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at wher...
*2025-03-17 | JKHY | Score: 0.484*

**4.** 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose ...
*2025-05-29 | NFLX | Score: 0.463*

**5.** Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at wher...
*2025-05-30 | META | Score: 0.461*



--------------------------------------------------------------------------------


# Question: What financial news headlines connect Amazon with automation or logistics?

## Answer:
Several financial news headlines connect Amazon with automation and logistics, highlighting the company's ongoing integration of technology into its operations and its competitive positioning in the logistics sector.

1. **Amazon's AI Roadmap**: An article featuring Matt Garman, CEO of Amazon Web Services, emphasizes that "every aspect of Amazon is leveraging artificial intelligence." This indicates a strong commitment to automation across its logistics and operational frameworks, suggesting that Amazon is continuously enhancing its efficiency and service delivery through AI technologies.

2. **UPS and Stord's Acquisition**: A significant development in the logistics landscape is UPS's sale of its subsidiary Ware2Go to Stord, a logistics tech startup. This acquisition is aimed at challenging Amazon's dominance in e-commerce logistics by expanding Stord's fulfillment capabilities. The article notes that the deal adds "2.5 million square feet of fulfillment space" to Stord's network, which is crucial for competing with Amazon's extensive logistics infrastructure.

3. **Market Dynamics in Logistics**: Another article discusses the competitive landscape of air freight and logistics stocks, indirectly referencing Amazon's influence in the sector. As Amazon continues to expand its logistics capabilities, it impacts the performance and strategies of other logistics firms, including C.H. Robinson Worldwide.

4. **AI and Automation in Earnings Growth**: Although not directly about Amazon, a report on Woodward highlights how "increasing volumes, pricing, automation, and products will push its aerospace margins." This context reflects a broader trend in which companies, including Amazon, are leveraging automation to drive growth and efficiency in logistics and operations.

These articles collectively illustrate Amazon's strategic focus on automation and logistics, showcasing its efforts to maintain a competitive edge through technological advancements and operational efficiencies.

## Sources:
**1.** Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazo...
*2025-05-25 | TFC | Score: 0.601*

**2.** Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects ...
*2025-05-30 | AMZN | Score: 0.523*

**3.** Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi...
*2025-05-23 | TFC | Score: 0.516*

**4.** Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ...
*2025-04-30 | AON | Score: 0.513*

**5.** Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of th...
*2025-05-23 | UPS | Score: 0.503*



--------------------------------------------------------------------------------


In [ ]:
test_rag_questions(rag, questions_industry, "Industry")

Testing Industry Questions:


# Question: What are the main themes emerging in financial news about the semiconductor industry?

## Answer:
The financial news surrounding the semiconductor industry reveals several key themes that are shaping investor sentiment and market dynamics:

1. **International Revenue Trends**: Articles highlight the importance of international revenue for semiconductor companies like ON Semiconductor Corp. (ON). The performance of these companies in global markets is increasingly influencing Wall Street forecasts and stock valuations. Investors are keenly assessing how these trends affect overall financial health and growth potential.

2. **Investor Attention and Stock Performance**: ON Semiconductor has been a focal point for investors, with multiple articles discussing its stock performance and the factors driving investor interest. Despite reporting soft earnings, ON's stock saw a significant price surge of 30% over a month, attributed to share buyback programs and investor confidence, indicating a market that is willing to overlook short-term losses for potential long-term gains.

3. **Earnings and Market Reactions**: The semiconductor sector is experiencing mixed earnings reports, with some companies like ON Semiconductor facing net losses while still attracting investor interest. This suggests a market environment where investors are looking beyond immediate financial results to future growth prospects, particularly in sectors like AI and electric vehicles (EVs).

4. **Sector-Specific Growth Areas**: There is a notable focus on specific growth areas within the semiconductor industry, such as silicon carbide (SiC) and AI data centers. Companies like ON are highlighted for their growth in these segments, even as demand for EVs faces challenges. This indicates a shift in focus towards technologies that are expected to drive future growth.

5. **Market Volatility and Economic Conditions**: The semiconductor industry is navigating challenging macroeconomic conditions, which are impacting demand across various sectors. Articles mention declining demand for EVs and the broader economic landscape as factors influencing stock performance and investor sentiment.

6. **Stock Picks and Investment Strategies**: The articles also discuss investment strategies, with mentions of notable investors like Glenn Russell Dubin selecting stocks with high upside potential. This reflects a broader trend of investors seeking out semiconductor stocks that are perceived to have strong growth potential despite current market volatility.

7. **Comparative Performance within the Industry**: The articles compare various semiconductor companies, such as Vishay Intertechnology and NXP Semiconductors, assessing their performance and potential as investment opportunities. This comparative analysis is crucial for investors looking to identify which companies may outperform others in the current economic climate.

In summary, the semiconductor industry is characterized by a complex interplay of international revenue dynamics, investor sentiment towards growth potential despite earnings challenges, sector-specific advancements, and the overarching influence of macroeconomic conditions. Investors are actively seeking opportunities in this space, particularly in promising technologies like AI and SiC, while navigating the volatility that comes with the current economic landscape.

## Sources:
**1.** Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall S...
*2025-05-13 | ON | Score: 0.643*

**2.** ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes...
*2025-05-21 | ON | Score: 0.582*

**3.** Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week......
*2025-05-12 | ON | Score: 0.565*

**4.** Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handl...
*2025-05-29 | ADI | Score: 0.548*

**5.** ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upsid...
*2025-05-11 | ON | Score: 0.528*



--------------------------------------------------------------------------------


# Question: What trends are being reported in the retail industry?

## Answer:
The retail industry is currently experiencing several notable trends, as highlighted in the recent financial news articles. Here are the key insights:

1. **Declining Stock Performance**: Retail stocks have faced significant declines over the past six months, with reports indicating a drop of approximately 13.7% in the retail sector, which is worse than the S&P 500's decline of 5.5% (Article 1). Another article notes that consumer discretionary businesses have also suffered, with a 12.3% decline, again outpacing the S&P 500's 2.1% loss (Article 3). This trend indicates a challenging environment for retailers, likely influenced by broader economic factors.

2. **Volatility in Consumer Demand**: The retail industry is highly sensitive to fluctuations in consumer spending. As economic conditions shift, demand for retail products can become unpredictable, leading to volatility in sales and stock performance (Article 1). This uncertainty has raised concerns among analysts regarding the sustainability of current retail business models.

3. **Supply Chain Adjustments**: Retailers are adapting to ongoing trade tensions and tariff policies by altering their supply chains. Executives have reported that many price increases have already been implemented in response to these changes, suggesting that retailers are proactively managing costs to mitigate the impact of tariffs (Article 2).

4. **Inventory Management Challenges**: There is a looming threat of inventory overflow due to a surge in imports ahead of a temporary tariff pause between the U.S. and China. Analysts warn that this could lead to deeper discounts and margin pressures for retailers, as they may need to clear excess stock (Article 16). This situation highlights the complexities of inventory management in a fluctuating market.

5. **Economic Sensitivity**: The performance of retail stocks is closely tied to economic cycles, with recent trends indicating that demand may be waning. Reports suggest that discretionary spending is decreasing, which could further exacerbate the challenges faced by retailers (Article 12). 

6. **Sector-Specific Challenges**: Certain segments within retail, such as restaurants, are also facing difficulties due to high inventory and labor costs, which can lead to thin margins. The restaurant sector has seen a decline of 13% over the past six months, indicating that the challenges are widespread across different retail categories (Article 6).

7. **Price Sensitivity and Consumer Behavior**: The grocery sector remains crucial for many retailers, as it drives store visits and customer loyalty. Even as some retailers raise prices, they are cautious about keeping grocery bills manageable to maintain customer engagement (Article 22). This reflects a strategic focus on balancing pricing with consumer expectations.

In summary, the retail industry is navigating a complex landscape characterized by declining stock performance, volatile consumer demand, supply chain adjustments, inventory management challenges, and economic sensitivity. Retailers are adapting their strategies to cope with these pressures, but the outlook remains uncertain as they face both internal and external challenges.

## Sources:
**1.** 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of...
*2025-05-12 | KMX | Score: 0.590*

**2.** Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increase...
*2025-05-29 | BBY | Score: 0.506*

**3.** 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working aga...
*2025-05-22 | HLT | Score: 0.500*

**4.** Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Among...
*2025-04-26 | PKG | Score: 0.476*

**5.** Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'...
*2025-05-14 | APD | Score: 0.476*



--------------------------------------------------------------------------------


# Question: What risks or challenges are discussed in recent news about the energy industry?

## Answer:
Recent news articles highlight several significant risks and challenges facing the energy industry, particularly in the context of both renewable and traditional energy sectors.

1. **Legislative Risks for Renewable Energy**: A major concern for the renewable energy sector is the potential repeal of critical subsidies. The U.S. House of Representatives has advanced a bill that could significantly undermine the economic viability of renewable energy projects by removing essential financial support. This legislative move has already resulted in a sharp decline in renewable energy stocks, as companies that had ramped up production may find their projects uneconomical if the bill becomes law (Articles 1 and 2). The uncertainty surrounding this legislation poses a substantial risk to investments in renewable energy.

2. **Economic Pressures on Oilfield Services**: The oilfield service sector, represented by companies like Schlumberger (SLB), Halliburton (HAL), and Baker Hughes (BKR), faces multiple challenges. These include declining oil prices, rising tariffs, and shrinking drilling budgets, which collectively threaten profitability and operational viability. The article discusses whether demand for liquefied natural gas (LNG) and advancements in artificial intelligence (AI) can provide enough support to offset these pressures (Articles 3 and 4). The combination of these factors creates a precarious environment for oilfield service companies.

3. **Impact of Tariffs and Trade Wars**: The ongoing trade conflicts, particularly those initiated during the Trump administration, have introduced significant uncertainty into the energy market. Tariffs on materials and equipment essential for energy production can increase costs and complicate supply chains, further straining the financial health of energy companies (Article 5). The unpredictability of the tariff landscape adds another layer of risk for investors and companies alike.

4. **Wildfire Liability and Regulatory Issues**: Companies like Xcel Energy are grappling with the financial implications of wildfire liabilities, which have become a growing concern in regions prone to such disasters. The emergence of new legal theories regarding liability could lead to increased litigation costs and regulatory scrutiny, impacting financial performance (Article 8). This highlights the operational risks that energy companies face in managing environmental and legal challenges.

5. **Market Volatility and Economic Downturns**: The broader economic environment is also affecting the energy sector. Industrial businesses, which are closely tied to energy production and consumption, have seen a decline in stock performance, indicating investor concerns about a potential economic downturn. The energy sector's performance is often linked to economic cycles, and a prolonged downturn could exacerbate existing challenges (Articles 9, 11, and 13).

6. **Specific Project Risks**: Projects like Dominion Energy's offshore wind initiative are at risk due to political opposition and regulatory hurdles. The future of such projects, which are critical for job creation and energy needs, remains uncertain amidst changing political landscapes (Article 7).

In summary, the energy industry is currently navigating a complex landscape filled with legislative risks, economic pressures, trade conflicts, environmental liabilities, and market volatility. These challenges could significantly impact the industry's growth and profitability in the near future.

## Sources:
**1.** Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
*2025-05-23 | NEE | Score: 0.536*

**2.** Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
*2025-05-23 | ENPH | Score: 0.536*

**3.** Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand o...
*2025-05-21 | HAL | Score: 0.482*

**4.** Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand o...
*2025-05-21 | BKR | Score: 0.482*

**5.** 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to cl...
*2025-05-21 | FCX | Score: 0.478*



--------------------------------------------------------------------------------


## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

YOUR WRITTEN RESPONSE HERE

The retrieved news snippets generally provide strong support for the generated answers. In the Topic section, particularly in the question about inflation, the answer references persistent inflation, food prices, and tariffs, which are all directly mentioned in the supporting context. Similarly, in the Company section, the summaries of Microsoft’s leadership in innovation and Amazon’s efforts in automation are well aligned with the retrieved snippets, which mention product strategies and logistical advancements. In the Industry section, key sectoral concerns such as AI data centers in semiconductors, inventory pressures in retail, and legislative risks in energy are clearly grounded in the retrieved excerpts. Overall, the LLM’s answers demonstrate solid semantic alignment with the context, suggesting that the retrieval component is functioning effectively in supporting grounded generation.

#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

YOUR WRITTEN RESPONSE HERE
  
The answers generally cover the main thrust of each question but often at a high level of abstraction. In the Topic set, the response on investor sentiment correctly identifies a mix of optimism and caution, but it misses some sector-specific nuances that appear in the retrieved context. Similarly, the AI in finance response identifies general trends such as improved operations and investment strategies, but it omits certain use cases or company-specific implementations that were present in the context. In the Industry set, answers highlight central issues but do not go deep into numbers, timeframes, or regional dynamics even when the context allows it. This suggests that while the LLM generates valid summaries, it sometimes leaves out more granular or specific insights that could make the responses more comprehensive and informative.

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

YOUR WRITTEN RESPONSE HERE

In general, there is little irrelevant information retrieved, and the context is mostly topically consistent with the question themes. However, in the Industry section, a few snippets contain broad market commentary not directly tied to the specific sectors being queried. For instance, vague statements about market volatility or global risk appear in contexts labeled for specific industries. While these snippets do not severely distort the answers, they introduce the risk of vague or generalized responses when more focused insights were available. This minor noise could potentially reduce the precision of the output if not properly managed, especially when the LLM defaults to synthesizing all retrieved inputs equally.


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

YOUR WRITTEN RESPONSE HERE

There are some minor inconsistencies across the retrieved snippets, particularly in topics involving market sentiment or forecasting. For example, in the investor sentiment question from the Topic section, some snippets reflect bullish outlooks while others are skeptical about analyst predictions. The LLM handles this appropriately by identifying the sentiment as “mixed” and acknowledging the existence of both optimism and caution. Similarly, in the Energy section of the Industry test, some articles emphasize the growth potential of renewables, while others warn about legislative and valuation risks. Rather than attempting to resolve these contradictions, the model presents a balanced perspective that integrates multiple viewpoints. This demonstrates an effective handling of conflict in context, especially in cases where no single narrative dominates the discourse.
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

YOUR WRITTEN RESPONSE HERE
  
There are several ways to improve both retrieval and generation quality. One area for enhancement is the filtering of retrieved documents using richer metadata, such as sector, date, or publisher credibility. This could help avoid the inclusion of snippets that are only marginally related to the core question. Additionally, incorporating metadata into the prompts could help the LLM better contextualize its generation. For example, explicitly stating that the answer should focus on a given industry or timeframe may guide the model toward more precise outputs. Increasing the number of retrieved documents (k) could be beneficial for more complex or comparative questions, though care must be taken to prevent additional noise. Improvements could also be made in how documents are chunked prior to embedding; semantic chunking rather than fixed-length token windows might yield more coherent retrieval units. Finally, adding structured few-shot examples or scaffolding in the prompt could help the model produce more detailed and balanced answers by clarifying expectations on tone, scope, or content structure.

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [ ]:
# CODE HERE
# Use as many coding cells as you need

df_enriched = df_news.merge(df_meta, on='TICKER', how='left')
df_enriched['COMPANY_NAME'] = df_enriched['COMPANY_NAME'].fillna('Unknown Company')
df_enriched['SECTOR'] = df_enriched['SECTOR'].fillna('Unknown Sector')
df_enriched['INDUSTRY'] = df_enriched['INDUSTRY'].fillna('Unknown Industry')

display(df_enriched.head(5))

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL,EMBEDDED_TEXT,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...,2 Dow Jones Stocks with Promising Prospects an...,3M Company,Industrials,Conglomerates
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...,3 S&P 500 Stocks Skating on Thin Ice : The S&P...,3M Company,Industrials,Conglomerates
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...,3M Rises 15.8% YTD: Should You Buy the Stock N...,3M Company,Industrials,Conglomerates
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,3M Company,Industrials,Conglomerates
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...,3 Cash-Producing Stocks with Questionable Fund...,3M Company,Industrials,Conglomerates


In [ ]:
enriched_metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER'],
        'COMPANY_NAME': row['COMPANY_NAME'],
        'SECTOR': row['SECTOR'],
        'INDUSTRY': row['INDUSTRY']
    }
    for _, row in df_enriched.iterrows()
]

In [ ]:
faiss_store_enriched = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=enriched_metadata
)

In [ ]:
class RAGv2:
    def __init__(self, vector_store, openai_client, k=K):
        self.vector_store = vector_store
        self.client = openai_client
        self.k = k

    def query(self, question):
        docs = self.vector_store.search(question, self.k)

        context = ""

        for i, (doc, meta, score) in enumerate(docs):
          context += f"""Article {i+1}:
            Content: {doc}
            Company: {meta['COMPANY_NAME']} ({meta['TICKER']})
            Sector: {meta['SECTOR']}
            Industry: {meta['INDUSTRY']}
            Date: {meta['PUBLICATION_DATE']}
            Relevance Score: {score:.3f}

            """

        prompt = f"""Question: {question}

          Based on the following financial news articles with company and industry context, provide a comprehensive and well-structured answer. Use the metadata (company names, sectors, industries) to provide more specific and contextual insights.

          Context Articles:
          {context}

            Answer:"""


        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a financial analyst. Answer based on the provided news articles AND their associated metadata. Be specific and cite details."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=800
        )

        answer = response.choices[0].message.content
        doc_info = [(doc, meta, score) for doc, meta, score in docs]

        return answer, doc_info

    def display_results(self, question, answer, docs):
        output = f"# Question: {question}\n\n## Answer:\n{answer}\n\n## Sources:\n"

        for i, (doc, meta, score) in enumerate(docs[:5], 1):
          output += f"**{i}.** {doc[:200]}...\n"
          output += f"*Company: {meta['COMPANY_NAME']} ({meta['TICKER']}) | "
          output += f"Sector: {meta['SECTOR']} | Industry: {meta['INDUSTRY']} | "
          output += f"Date: {meta['PUBLICATION_DATE']} | Score: {score:.3f}*\n\n"

        print_markdown(output)

In [ ]:
rag_enriched = RAGv2(faiss_store_enriched, client)

In [ ]:
test_rag_questions(rag_enriched, questions_topic, "Topic")

Testing Topic Questions:


# Question: What are the major concerns expressed in financial news about inflation?

## Answer:
The financial news articles highlight several major concerns regarding inflation and its implications for various sectors and companies. Here’s a structured overview of these concerns:

### 1. **Persistent Inflation Risks**
The Federal Reserve's minutes from their May policy meeting indicate a growing concern over persistent inflation in the U.S. This sentiment is echoed in Article 1, where BlackRock, Inc. (BLK), a key player in the financial services sector, is closely monitoring these inflationary pressures. The Fed's apprehension suggests that inflation could lead to an economic slowdown, which has implications for asset management and investment strategies.

### 2. **Food Inflation Impact**
Articles 2, 3, 4, and 6 collectively discuss the impact of food inflation on consumer sentiment and economic policy. For instance, Tesla, Inc. (TSLA) and lululemon athletica inc. (LULU), both in the consumer cyclical sector, are facing challenges due to rising food prices, which dampen hopes for interest rate cuts. This inflationary trend in essential goods is affecting consumer discretionary spending, which could lead to reduced sales for companies reliant on consumer spending.

### 3. **Tariffs and Supply Chain Disruptions**
Article 2 also mentions "tariff twists and turns," indicating that ongoing trade tensions and tariffs are exacerbating inflationary pressures. Companies like Broadcom Inc. (AVGO) and NVIDIA Corporation (NVDA), both in the technology sector, are likely to feel the effects of increased costs associated with tariffs on imported goods, which could impact their profit margins and pricing strategies.

### 4. **Macroeconomic Uncertainty**
Article 19 discusses a grim economic outlook that overshadows earnings reports, suggesting that macroeconomic uncertainty, driven by inflation and other factors, is affecting investor sentiment. This uncertainty is reflected in the performance of consumer cyclical stocks, which have seen a significant drawdown, as noted in Articles 16 and 25. Companies like CarMax, Inc. (KMX) and Hasbro, Inc. (HAS) are particularly vulnerable as consumer spending habits shift in response to rising prices.

### 5. **Fiscal Health Concerns**
Article 14 highlights the rising U.S. national debt and its implications for inflation and fiscal sustainability. With interest payments on the national debt approaching $1 trillion, there are concerns about long-term economic stability. This situation is prompting investors to seek refuge in hard assets like gold, as indicated by Newmont Corporation (NEM), which operates in the gold sector.

### 6. **Earnings Outlook Adjustments**
Finally, Article 15 points to a downward shift in earnings expectations across various sectors due to the uncertain macroeconomic environment. Companies are adjusting their forecasts in light of inflationary pressures, which could lead to lower stock valuations and increased volatility in the market.

### Conclusion
In summary, the major concerns regarding inflation expressed in the financial news revolve around persistent inflation risks, the impact of food prices, tariff-related supply chain disruptions, macroeconomic uncertainty, fiscal health issues, and adjustments in earnings outlooks. These factors collectively indicate a challenging environment for businesses across various sectors, particularly those in consumer cyclical and technology industries, as they navigate the complexities of inflation and its broader economic implications.

## Sources:
**1.** Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown....
*Company: BlackRock, Inc. (BLK) | Sector: Financial Services | Industry: Asset Management | Date: 2025-05-29 | Score: 0.577*

**2.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*Company: Tesla, Inc. (TSLA) | Sector: Consumer Cyclical | Industry: Auto Manufacturers | Date: 2025-05-31 | Score: 0.492*

**3.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*Company: NVIDIA Corporation (NVDA) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-31 | Score: 0.492*

**4.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*Company: lululemon athletica inc. (LULU) | Sector: Consumer Cyclical | Industry: Apparel Retail | Date: 2025-05-31 | Score: 0.492*

**5.** The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
*Company: Broadcom Inc. (AVGO) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-31 | Score: 0.492*



--------------------------------------------------------------------------------


# Question: How is investor sentiment described in recent financial headlines?

## Answer:
Investor sentiment in recent financial headlines reflects a complex landscape characterized by both optimism and caution. Several articles highlight a prevailing bullishness among analysts regarding various stocks, yet they also caution investors to remain skeptical of these optimistic forecasts due to potential biases and institutional pressures.

1. **Optimism Amidst Skepticism**: Many articles emphasize that Wall Street is overwhelmingly bullish on certain stocks, suggesting significant upside potential. For instance, companies like **CarMax, Inc. (KMX)** and **Monolithic Power Systems, Inc. (MPWR)** have attracted attention with price targets implying returns above 20%. However, analysts warn that such forecasts should be taken with caution, as they may be influenced by the desire to maintain favorable relationships with the companies they cover. This sentiment is echoed in multiple articles, including those discussing **Revvity, Inc. (RVTY)** and **Expedia Group, Inc. (EXPE)**, where analysts are hesitant to issue sell ratings due to fears of losing business opportunities in other areas like M&A advisory.

2. **Catalysts Driving Performance**: Positive catalysts are noted as significant drivers of stock performance. For example, **Microchip Technology Incorporated (MCHP)** and **News Corporation (NWS)** have seen their stocks outperform the market due to new product launches and positive media coverage. This suggests that investor sentiment is buoyed by tangible developments that enhance the perceived value of these companies.

3. **Cautionary Signals**: Despite the bullish sentiment, there are notable warnings about stocks facing headwinds. Articles discussing companies like **Darden Restaurants, Inc. (DRI)** and **Caterpillar Inc. (CAT)** highlight bearish forecasts that stand out in an environment where analysts typically avoid issuing negative ratings. This indicates a growing concern among investors about certain sectors, particularly within the consumer cyclical and industrials sectors, where economic pressures may be mounting.

4. **Contrasting Views on Popular vs. Unpopular Stocks**: The sentiment is further complicated by the distinction between popular stocks, which analysts are generally optimistic about, and those that are deemed unpopular or facing challenges. For instance, stocks like **McCormick & Company, Incorporated (MKC)** are described as having "questionable fundamentals," indicating that while they may have been favored in the past, current sentiment is shifting towards caution.

5. **Market Dynamics**: The overall investor sentiment appears to be influenced by broader market dynamics, including the performance of specific sectors like technology and healthcare, where companies like **Align Technology, Inc. (ALGN)** and **Salesforce (ORCL)** are discussed in the context of their recent earnings surprises and the lingering presence of bearish sentiment.

In summary, while there is a notable bullish sentiment towards several stocks, particularly in the technology and healthcare sectors, caution is advised due to potential biases in analyst ratings and the presence of stocks facing significant challenges. Investors are encouraged to critically assess the underlying fundamentals and market conditions before making investment decisions.

## Sources:
**1.** 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to rem...
*Company: CarMax, Inc. (KMX) | Sector: Consumer Cyclical | Industry: Auto & Truck Dealerships | Date: 2025-05-26 | Score: 0.612*

**2.** 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new pro...
*Company: Microchip Technology Incorporated (MCHP) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-20 | Score: 0.598*

**3.** 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying ret...
*Company: Monolithic Power Systems, Inc. (MPWR) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-06 | Score: 0.589*

**4.** 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue g...
*Company: Darden Restaurants, Inc. (DRI) | Sector: Consumer Cyclical | Industry: Restaurants | Date: 2025-05-21 | Score: 0.577*

**5.** 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. How...
*Company: Revvity, Inc. (RVTY) | Sector: Healthcare | Industry: Diagnostics & Research | Date: 2025-05-23 | Score: 0.570*



--------------------------------------------------------------------------------


# Question: What role is artificial intelligence playing in recent finance-related news stories?

## Answer:
Artificial intelligence (AI) is increasingly becoming a pivotal force in the finance and technology sectors, as evidenced by a range of recent news articles highlighting its diverse applications and implications for various companies. Here’s a structured overview of the role AI is playing based on the provided articles:

### 1. **Enhancing Financial Services and Lending**
- **Jack Henry & Associates, Inc. (JKHY)** is integrating AI-driven lending technology with Algebrik to enhance productivity and reduce human error in financial services. This indicates a trend where financial technology companies are leveraging AI to streamline operations and improve customer experiences in lending processes (Article 1).

- **Upstart**, mentioned in the context of hedge fund investments, is utilizing AI to help lenders quantify credit risk effectively. This application of AI not only aids in risk assessment but also allows for more attractive returns for lenders, showcasing how AI can transform traditional lending models (Article 3).

### 2. **Investment and Market Dynamics**
- **Meta Platforms, Inc. (META)** is making significant investments in AI applications, which are expected to enhance its stock value beyond its legacy business. The focus on AI is seen as a critical factor for future growth, as investors increasingly look for companies that can leverage AI to gain competitive advantages (Article 2).

- **Palantir Technologies Inc. (PLTR)** has been highlighted multiple times for its strong performance in the AI sector, particularly in government and commercial applications. The company’s AI platform is designed to integrate generative AI capabilities into operations, which has led to substantial stock gains and investor interest (Articles 4, 9, 24).

### 3. **AI in Software and Application Development**
- **Salesforce, Inc. (CRM)** has faced scrutiny for its heavy focus on AI, suggesting that while AI can drive innovation, over-reliance may lead to challenges in other business areas. This reflects a broader concern in the tech industry about balancing AI investments with overall business health (Article 7).

- **Intuit Inc. (INTU)** is leveraging AI to improve the taxpayer experience, indicating that AI can enhance customer service and operational efficiency in financial software applications (Article 21).

### 4. **Market Sentiment and Stock Performance**
- The performance of AI-related stocks is closely monitored, with companies like **Nvidia** and **Palantir** being at the forefront of this trend. Analysts are optimistic about the potential for AI stocks to generate significant returns, with some expecting gains as high as 240% for certain stocks (Article 3).

- The **Gartner CFO conference** highlighted the shifting priorities in finance, emphasizing practical AI adoption and its implications for financial leaders. This suggests that AI is not only a technological trend but also a strategic consideration for finance professionals (Article 12).

### 5. **Broader Implications for the Industry**
- The AI data analytics market is projected to grow significantly, with companies like **BigBear.ai** and **Palantir** competing for market share in this burgeoning field. The focus on AI analytics reflects a shift towards data-driven decision-making in various sectors, including defense and commercial industries (Articles 19, 20).

In conclusion, AI is playing a transformative role across the finance and technology sectors by enhancing operational efficiencies, driving investment strategies, and reshaping market dynamics. Companies are increasingly recognizing the necessity of integrating AI into their business models to remain competitive and meet evolving consumer demands. This trend is likely to continue as AI technology matures and its applications expand.

## Sources:
**1.** Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at wher...
*Company: Jack Henry & Associates, Inc. (JKHY) | Sector: Technology | Industry: Information Technology Services | Date: 2025-03-17 | Score: 0.697*

**2.** This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy busin...
*Company: Meta Platforms, Inc. (META) | Sector: Communication Services | Industry: Internet Content & Information | Date: 2025-05-31 | Score: 0.626*

**3.** Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in th...
*Company: Palantir Technologies Inc. (PLTR) | Sector: Technology | Industry: Software - Infrastructure | Date: 2025-05-31 | Score: 0.619*

**4.** Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stoc...
*Company: Palantir Technologies Inc. (PLTR) | Sector: Technology | Industry: Software - Infrastructure | Date: 2025-05-31 | Score: 0.618*

**5.** 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose ...
*Company: Netflix, Inc. (NFLX) | Sector: Communication Services | Industry: Entertainment | Date: 2025-05-29 | Score: 0.580*



--------------------------------------------------------------------------------


In [ ]:
test_rag_questions(rag_enriched, questions_company, "Company")

Testing Company Questions:


# Question: How is Microsoft being portrayed in news stories about artificial intelligence?

## Answer:
In recent news articles, Microsoft Corporation (MSFT) is portrayed as a significant player in the artificial intelligence (AI) sector, with a focus on its strategic partnerships and innovations in AI technology. The coverage highlights Microsoft's involvement in AI-driven initiatives and collaborations, particularly with other tech giants.

1. **Strategic Collaborations**: Microsoft is featured prominently in discussions about partnerships in the AI landscape. For instance, in an article discussing ServiceNow's resurgence, Microsoft is mentioned as one of the key players collaborating with ServiceNow and Amazon in AI deals. This indicates Microsoft's active role in leveraging AI technologies to enhance its service offerings and strengthen its market position (Article 21, 22).

2. **Market Positioning**: The articles suggest that Microsoft is positioned as a leader in the AI revolution, alongside other major companies like Google and NVIDIA. The AI market is projected to experience substantial growth, and Microsoft's innovations are seen as pivotal in driving this expansion. The article on AI chips notes that Microsoft is among the major players leading innovations in this space, which is crucial as the demand for AI capabilities continues to surge (Article 6).

3. **Financial Implications**: While Microsoft is recognized for its advancements in AI, there are also mentions of concerns regarding its market performance. An article discusses how Marvell's stock slide could impact Microsoft's position in the AI chip market, suggesting that there are competitive pressures that Microsoft must navigate (Article 7). This reflects a nuanced view of Microsoft’s standing, acknowledging both its leadership role and the challenges it faces.

4. **Broader Industry Context**: The portrayal of Microsoft is set against a backdrop of a rapidly evolving AI landscape, where companies are racing to capitalize on AI technologies. The articles collectively illustrate a competitive environment where Microsoft is striving to maintain its edge through innovation and strategic partnerships, while also facing scrutiny regarding its market strategies and performance.

Overall, Microsoft is depicted as a central figure in the AI sector, characterized by its proactive engagement in partnerships and innovation, while also contending with competitive pressures that could affect its market standing. This duality in portrayal highlights both the opportunities and challenges that Microsoft faces in the burgeoning AI market.

## Sources:
**1.** This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy busin...
*Company: Meta Platforms, Inc. (META) | Sector: Communication Services | Industry: Internet Content & Information | Date: 2025-05-31 | Score: 0.543*

**2.** How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. L...
*Company: Salesforce, Inc. (CRM) | Sector: Technology | Industry: Software - Application | Date: 2025-05-29 | Score: 0.505*

**3.** Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at wher...
*Company: Jack Henry & Associates, Inc. (JKHY) | Sector: Technology | Industry: Information Technology Services | Date: 2025-03-17 | Score: 0.484*

**4.** 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose ...
*Company: Netflix, Inc. (NFLX) | Sector: Communication Services | Industry: Entertainment | Date: 2025-05-29 | Score: 0.463*

**5.** Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at wher...
*Company: Meta Platforms, Inc. (META) | Sector: Communication Services | Industry: Internet Content & Information | Date: 2025-05-30 | Score: 0.461*



--------------------------------------------------------------------------------


# Question: What financial news headlines connect Amazon with automation or logistics?

## Answer:
Several financial news articles highlight the connection between Amazon and the themes of automation and logistics, showcasing the company's strategic initiatives and competitive landscape.

1. **Amazon's AI Roadmap**: In an article featuring Matt Garman, the CEO of Amazon Web Services (AWS), it is noted that "every aspect of Amazon is leveraging artificial intelligence." This indicates that Amazon is not only focusing on e-commerce but is also integrating AI into its logistics and operational processes, which is crucial for enhancing efficiency and automating various functions within its supply chain (Article 2, dated May 30, 2025).

2. **Stord's Acquisition of Ware2Go**: Another significant development is the acquisition of Ware2Go, a subsidiary of United Parcel Service (UPS), by the logistics tech startup Stord. This move is aimed at challenging Amazon's dominance in e-commerce logistics by expanding Stord's fulfillment capabilities. The article mentions that this acquisition adds 2.5 million square feet of fulfillment space to Stord's network, which is a direct competitive response to Amazon's extensive logistics infrastructure (Article 8, dated May 26, 2025).

3. **Earnings Reports in Logistics Sector**: A comparative analysis of logistics companies, including C.H. Robinson Worldwide, highlights the competitive dynamics in the air freight and logistics sector, where Amazon is a key player. The article discusses the performance of logistics stocks and indirectly emphasizes Amazon's influence in shaping market trends within this industry (Articles 5 and 6, both dated May 23, 2025).

4. **UPS and Amazon's Competitive Landscape**: The article discussing UPS's strategic moves notes the increasing attention on the logistics sector, particularly in the context of Amazon's market presence. UPS's initiatives are often viewed through the lens of competition with Amazon, which continues to innovate in logistics and delivery systems (Article 22, dated May 22, 2025).

These articles collectively illustrate how Amazon is leveraging automation and AI to enhance its logistics capabilities while simultaneously facing competitive pressures from other logistics firms aiming to capture market share in the e-commerce space. The integration of advanced technologies in logistics is a key theme that connects Amazon with broader industry trends.

## Sources:
**1.** Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazo...
*Company: Truist Financial Corporation (TFC) | Sector: Financial Services | Industry: Banks - Regional | Date: 2025-05-25 | Score: 0.601*

**2.** Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects ...
*Company: Amazon.com, Inc. (AMZN) | Sector: Consumer Cyclical | Industry: Internet Retail | Date: 2025-05-30 | Score: 0.523*

**3.** Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi...
*Company: Truist Financial Corporation (TFC) | Sector: Financial Services | Industry: Banks - Regional | Date: 2025-05-23 | Score: 0.516*

**4.** Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ...
*Company: Aon plc (AON) | Sector: Financial Services | Industry: Insurance Brokers | Date: 2025-04-30 | Score: 0.513*

**5.** Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of th...
*Company: United Parcel Service, Inc. (UPS) | Sector: Industrials | Industry: Integrated Freight & Logistics | Date: 2025-05-23 | Score: 0.503*



--------------------------------------------------------------------------------


In [ ]:
test_rag_questions(rag_enriched, questions_industry, "Industry")

Testing Industry Questions:


# Question: What are the main themes emerging in financial news about the semiconductor industry?

## Answer:
The financial news surrounding the semiconductor industry reveals several key themes that are shaping investor sentiment and market dynamics. These themes can be categorized into revenue trends, stock performance, and the impact of macroeconomic conditions.

### 1. **Revenue Trends and International Exposure**
A significant focus has been on the international revenue trends of semiconductor companies, particularly ON Semiconductor Corporation (ON). Articles emphasize the importance of assessing international revenues as a predictor of future performance and stock valuation. For instance, Article 1 discusses how ON's international revenue trends are crucial for Wall Street's forecasts, indicating that global market dynamics are increasingly influencing local stock performance. Similarly, Article 22 highlights Teradyne, Inc. (TER) and its overseas revenue trends, suggesting that international markets are vital for growth in the semiconductor sector.

### 2. **Stock Performance Amidst Earnings Fluctuations**
Despite facing challenges such as soft earnings, ON Semiconductor has seen a notable increase in stock price, with a reported 30% surge over the last month despite a net loss in Q1 2025 (Article 9). This paradox of rising stock prices amidst poor earnings performance suggests a strong investor confidence driven by share buyback programs and potential long-term growth prospects, particularly in sectors like Silicon Carbide (SiC) and AI data centers. Articles also indicate that ON's stock is under scrutiny, with discussions on whether it represents a buy, sell, or hold opportunity (Article 19).

### 3. **Market Sentiment and Investment Interest**
Investor interest in semiconductor stocks is robust, as seen in the attention ON Semiconductor is garnering from users on platforms like Zacks.com (Article 2). The mention of billionaire investors, such as Glenn Russell Dubin, selecting ON as a stock with significant upside potential (Article 5) reflects a broader trend where high-profile endorsements can influence market sentiment and attract retail investors.

### 4. **Sector-Specific Challenges and Opportunities**
The semiconductor industry is navigating a complex landscape characterized by declining demand in certain segments, such as electric vehicles (EVs), while simultaneously experiencing growth in others, such as AI and data centers (Article 19). This duality presents both challenges and opportunities for companies like ON and NXP Semiconductors (NXPI), which are highlighted for their potential to rebound based on specific market segments (Article 24).

### 5. **Technological Innovations and Market Drivers**
Technological advancements and the demand for new products are critical drivers of growth within the semiconductor sector. Articles mention companies like Analog Devices, Inc. (ADI) and Vishay Intertechnology (VSH), which are spotlighted for their performance in the analog semiconductor space (Article 4). The emphasis on AI and chip demand as a catalyst for growth further underscores the industry's reliance on innovation to sustain momentum.

### Conclusion
In summary, the semiconductor industry is currently characterized by a mix of optimism and caution. Key themes include the significance of international revenue trends, stock performance in light of earnings fluctuations, strong investor interest, and the dual challenges and opportunities presented by market dynamics. As the industry continues to evolve, companies that can effectively navigate these trends and leverage technological advancements are likely to emerge as leaders in the sector.

## Sources:
**1.** Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall S...
*Company: ON Semiconductor Corporation (ON) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-13 | Score: 0.643*

**2.** ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes...
*Company: ON Semiconductor Corporation (ON) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-21 | Score: 0.582*

**3.** Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week......
*Company: ON Semiconductor Corporation (ON) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-12 | Score: 0.565*

**4.** Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handl...
*Company: Analog Devices, Inc. (ADI) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-29 | Score: 0.548*

**5.** ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upsid...
*Company: ON Semiconductor Corporation (ON) | Sector: Technology | Industry: Semiconductors | Date: 2025-05-11 | Score: 0.528*



--------------------------------------------------------------------------------


# Question: What trends are being reported in the retail industry?

## Answer:
The retail industry is currently experiencing several notable trends, as highlighted by recent articles and their associated metadata. Here’s a structured overview of the key trends affecting the sector:

### 1. **Volatility in Consumer Demand**
The retail sector is facing significant volatility in consumer spending, which has led to a downturn in retail stock performance. For instance, the retail industry has seen a decline of approximately **13.7%** over the past six months, which is notably worse than the **5.5%** loss of the S&P 500 (Article 1). This decline indicates that consumer discretionary spending is under pressure, likely due to economic uncertainties.

### 2. **Impact of Trade Policies**
Retailers are adapting their supply chains in response to ongoing trade tensions, particularly those stemming from tariff policies. Executives from companies like Best Buy (BBY) have reported that many price increases have already been implemented due to tariffs, suggesting that retailers are proactively managing their costs and pricing strategies to mitigate the impact of trade disruptions (Article 2).

### 3. **Inventory Management Challenges**
There is a growing concern regarding inventory management, particularly in light of recent tariff pauses that have led to an influx of imports. Retailers such as Capri Holdings (CPRI) and Lululemon (LULU) may face inventory gluts, which could force them to offer deeper discounts, thus pressuring profit margins (Article 16). This trend underscores the challenges retailers face in balancing supply with fluctuating consumer demand.

### 4. **Performance of Consumer Discretionary Stocks**
The performance of consumer discretionary stocks is closely tied to economic cycles. Recent reports indicate that these stocks have pulled back by **12.3%** over the last six months, again worse than the S&P 500’s decline (Article 3). This trend suggests that as economic conditions tighten, consumers may prioritize essential spending over discretionary purchases.

### 5. **Pressure on Margins in the Restaurant Sector**
The restaurant industry, a significant segment of the retail sector, has also been under pressure, with a **13%** decline in stock performance over the past six months (Article 6). High inventory and labor costs contribute to thin margins, making it difficult for restaurants to maintain profitability during periods of reduced consumer spending.

### 6. **Discount Retailers and Grocery Segments**
Discount retailers like Walmart (WMT) and Costco (COST) continue to play a vital role in the retail landscape, particularly in the grocery segment, which drives store visits and customer loyalty (Article 22). Despite rising prices, these retailers are focusing on keeping grocery bills manageable to retain customer loyalty, indicating a strategic emphasis on value during economic uncertainty.

### 7. **Market Sentiment and Stock Valuation**
There is a growing sentiment among investors regarding the potential for retail stocks to be undervalued or viewed as traps. Analysts are scrutinizing stocks like Target (TGT) and others for their potential recovery or continued decline, reflecting a cautious approach in the market (Article 10). 

### Conclusion
In summary, the retail industry is navigating a complex landscape characterized by volatile consumer demand, the impact of trade policies, inventory management challenges, and pressure on profit margins, especially in the restaurant sector. Discount retailers are strategically positioned to leverage grocery sales to maintain customer loyalty amidst rising prices. As the market continues to evolve, investors remain cautious, weighing the potential for recovery against the backdrop of economic uncertainty.

## Sources:
**1.** 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of...
*Company: CarMax, Inc. (KMX) | Sector: Consumer Cyclical | Industry: Auto & Truck Dealerships | Date: 2025-05-12 | Score: 0.590*

**2.** Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increase...
*Company: Best Buy Co., Inc. (BBY) | Sector: Consumer Cyclical | Industry: Specialty Retail | Date: 2025-05-29 | Score: 0.506*

**3.** 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working aga...
*Company: Hilton Worldwide Holdings Inc. (HLT) | Sector: Consumer Cyclical | Industry: Lodging | Date: 2025-05-22 | Score: 0.500*

**4.** Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Among...
*Company: Packaging Corporation of America (PKG) | Sector: Consumer Cyclical | Industry: Packaging & Containers | Date: 2025-04-26 | Score: 0.476*

**5.** Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'...
*Company: Air Products and Chemicals, Inc. (APD) | Sector: Basic Materials | Industry: Specialty Chemicals | Date: 2025-05-14 | Score: 0.476*



--------------------------------------------------------------------------------


# Question: What risks or challenges are discussed in recent news about the energy industry?

## Answer:
Recent news articles highlight several significant risks and challenges facing the energy industry, particularly in the context of renewable energy and oil and gas sectors.

### 1. Legislative Risks for Renewable Energy
A major concern for the renewable energy sector is the potential repeal of critical subsidies. Articles 1 and 2 discuss how the U.S. House of Representatives is advancing a bill that could severely impact companies like **NextEra Energy, Inc. (NEE)** and **Enphase Energy, Inc. (ENPH)**. The repeal of subsidies may render many renewable projects uneconomical, jeopardizing investments and production ramp-ups that have been made in recent years. This legislative risk poses a significant threat to the viability of the renewable energy industry in the U.S., potentially leading to a crash in renewable energy stocks.

### 2. Economic Pressures on Oilfield Services
The oil and gas sector is also facing challenges, particularly for companies like **Halliburton Company (HAL)** and **Baker Hughes Company (BKR)**, as discussed in Articles 3 and 4. These companies are grappling with sliding oil prices, rising tariffs, and shrinking drilling budgets. The combination of these factors creates a tough operating environment, raising concerns about their profitability and future growth. The potential for demand in liquefied natural gas (LNG) and artificial intelligence (AI) to provide some support is uncertain, adding to the industry's volatility.

### 3. Wildfire Liability and Regulatory Challenges
Utilities such as **Xcel Energy Inc. (XEL)** and **Dominion Energy, Inc. (D)** are facing increasing scrutiny and regulatory challenges related to wildfire risks. Article 8 discusses Xcel's efforts to limit wildfire liability amid emerging litigation theories, while Article 7 highlights the precarious future of Dominion's offshore wind project due to political pressures. These challenges not only threaten operational stability but also expose these companies to potential financial liabilities and reputational risks.

### 4. Broader Economic Concerns
The energy sector is also affected by broader economic trends, as noted in Articles 9, 11, and 13, which discuss the industrial sector's struggles. The energy industry, particularly oil and gas, is closely tied to economic cycles, and a downturn could lead to reduced demand for energy services and products. The overall sentiment in the market suggests a prolonged downturn, which could further exacerbate the challenges faced by energy companies.

### 5. Tariff Impacts and Trade Wars
The ongoing trade conflicts, particularly those initiated during the Trump administration, have created an uncertain tariff landscape that impacts companies across the energy sector. Article 5 mentions how tariffs on critical materials like copper could affect the industrial economy, which in turn influences energy production and infrastructure development. Companies like **Freeport-McMoRan Inc. (FCX)** are particularly vulnerable to these changes, as they rely on stable trade conditions for their operations.

### Conclusion
In summary, the energy industry is navigating a complex landscape filled with legislative risks, economic pressures, regulatory challenges, and the impacts of trade wars. Companies in both the renewable and traditional energy sectors must adapt to these challenges to maintain profitability and ensure sustainable growth. The situation remains fluid, and stakeholders must stay informed about ongoing developments that could further influence the industry's trajectory.

## Sources:
**1.** Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
*Company: NextEra Energy, Inc. (NEE) | Sector: Utilities | Industry: Utilities - Regulated Electric | Date: 2025-05-23 | Score: 0.536*

**2.** Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
*Company: Enphase Energy, Inc. (ENPH) | Sector: Technology | Industry: Solar | Date: 2025-05-23 | Score: 0.536*

**3.** Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand o...
*Company: Halliburton Company (HAL) | Sector: Energy | Industry: Oil & Gas Equipment & Services | Date: 2025-05-21 | Score: 0.482*

**4.** Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand o...
*Company: Baker Hughes Company (BKR) | Sector: Energy | Industry: Oil & Gas Equipment & Services | Date: 2025-05-21 | Score: 0.482*

**5.** 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to cl...
*Company: Freeport-McMoRan Inc. (FCX) | Sector: Basic Materials | Industry: Copper | Date: 2025-05-21 | Score: 0.478*



--------------------------------------------------------------------------------


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | Clear structure with numbered themes and good organization.     | Enhanced clarity with specific company context and sector-specific insights that reduce ambiguity.   |
| Detail & Depth         | Good thematic coverage but lacks company-specific context.     | Richer detail with company names, sectors, and industry-specific examples providing deeper insights.   |
| Use of Context        | Generic use of article content without company identification.     | Strategic use of metadata to connect themes to specific companies and sectors for better context.   |
| Accuracy & Grounding       | Well-grounded in source material but misses company-specific nuances.     | More accurate with precise company identification and sector-specific analysis.   |
| Relevance      | Relevant to broad themes but may include less targeted information.     | Higher relevance through metadata filtering and company-sector alignment.   |
| Narrative Flow      | Smooth thematic progression but somewhat generic.     | Better narrative coherence linking companies, sectors, and themes into cohesive story.   |